In [1]:
import operator as op
import subprocess as sp
import glob
import os
from concurrent.futures import ThreadPoolExecutor
from itertools import chain

import tqdm
import numpy as np
import pandas as pd
from fn import F


In [2]:
os.chdir("/home/is6/work/cas_thesis/new_domains")

In [18]:
EVALUE = 10**(-18)

hmmscan_template = (
    f'hmmscan --noali --notextw --acc -E {EVALUE} --cpu 1 '
    f'-o {{output}} db.hmm {{proteins}}'
)

In [19]:
samples = pd.read_csv('samples.tsv', sep='\t')
samples.head()

,id,species
0,GCF_003294895.1,Aeromonas caviae
1,GCF_900636735.1,Pseudomonas aeruginosa
2,GCF_000832445.1,Bacillus anthracis
3,GCF_003432305.1,Escherichia coli
4,GCF_003864095.1,Escherichia albertii


In [20]:
# subsample assemblies

np.random.seed(139)  # set random seed for reproducibility 
k = 5  # the maximum number of assemblies per species
subsample = (
    samples
    .groupby('species')
    .apply(lambda grp:
        grp if len(grp) <= k else
        grp.sample(k)
    )
    .reset_index(drop=True, level=0)
)
subsample.shape

(5961, 2)

In [21]:
subsample.to_csv('samples_subsample.tsv', sep='\t', index=False)

In [7]:
os.makedirs('hmmscan', exist_ok=True)

In [8]:
ids = subsample['id']
protein_paths = [glob.glob(f'proteins/{id_}.faa')[0] for id_ in ids]
output_paths = [f'hmmscan/{id_}.txt' for id_ in ids]


In [13]:
protein_paths

['proteins/GCF_004214875.1.faa',
 'proteins/GCF_000196515.1.faa',
 'proteins/GCF_000018105.1.faa',
 'proteins/GCF_002005445.1.faa',
 'proteins/GCF_001766255.1.faa',
 'proteins/GCF_002173775.1.faa',
 'proteins/GCF_001766235.1.faa',
 'proteins/GCF_001499675.1.faa',
 'proteins/GCF_003966365.1.faa',
 'proteins/GCF_002220195.1.faa',
 'proteins/GCF_001628715.1.faa',
 'proteins/GCF_002202135.1.faa',
 'proteins/GCF_000010905.1.faa',
 'proteins/GCF_000010945.1.faa',
 'proteins/GCF_000241585.2.faa',
 'proteins/GCF_000010845.1.faa',
 'proteins/GCF_002006565.1.faa',
 'proteins/GCF_003391275.1.faa',
 'proteins/GCF_002456135.1.faa',
 'proteins/GCF_001499615.1.faa',
 'proteins/GCF_002549835.1.faa',
 'proteins/GCF_004843165.1.faa',
 'proteins/GCF_004843345.1.faa',
 'proteins/GCF_000247605.1.faa',
 'proteins/GCF_000144695.1.faa',
 'proteins/GCF_000266925.1.faa',
 'proteins/GCF_900660745.1.faa',
 'proteins/GCF_000967915.1.faa',
 'proteins/GCF_900660755.1.faa',
 'proteins/GCF_900476025.1.faa',
 'proteins

In [14]:
output_paths

['hmmscan/GCF_004214875.1.txt',
 'hmmscan/GCF_000196515.1.txt',
 'hmmscan/GCF_000018105.1.txt',
 'hmmscan/GCF_002005445.1.txt',
 'hmmscan/GCF_001766255.1.txt',
 'hmmscan/GCF_002173775.1.txt',
 'hmmscan/GCF_001766235.1.txt',
 'hmmscan/GCF_001499675.1.txt',
 'hmmscan/GCF_003966365.1.txt',
 'hmmscan/GCF_002220195.1.txt',
 'hmmscan/GCF_001628715.1.txt',
 'hmmscan/GCF_002202135.1.txt',
 'hmmscan/GCF_000010905.1.txt',
 'hmmscan/GCF_000010945.1.txt',
 'hmmscan/GCF_000241585.2.txt',
 'hmmscan/GCF_000010845.1.txt',
 'hmmscan/GCF_002006565.1.txt',
 'hmmscan/GCF_003391275.1.txt',
 'hmmscan/GCF_002456135.1.txt',
 'hmmscan/GCF_001499615.1.txt',
 'hmmscan/GCF_002549835.1.txt',
 'hmmscan/GCF_004843165.1.txt',
 'hmmscan/GCF_004843345.1.txt',
 'hmmscan/GCF_000247605.1.txt',
 'hmmscan/GCF_000144695.1.txt',
 'hmmscan/GCF_000266925.1.txt',
 'hmmscan/GCF_900660745.1.txt',
 'hmmscan/GCF_000967915.1.txt',
 'hmmscan/GCF_900660755.1.txt',
 'hmmscan/GCF_900476025.1.txt',
 'hmmscan/GCF_000018785.1.txt',
 'hmmsca

In [16]:
ls

assemblies@  db.hmm.h3m              hmms.ipynb         samples_subsample.tsv
bacteria@    db.hmm.h3p              md5checksums.txt@  samples.tsv
db.hmm       extract_proteins.ipynb  proteins/
db.hmm.h3f   figures/                README.txt@
db.hmm.h3i   hmmscan/                report.txt@


In [17]:
def hmmscan(input_path, output_path):
    command = hmmscan_template.format(proteins=input_path, output=output_path)
    return sp.run(command, check=True, shell=True)


In [22]:
with ThreadPoolExecutor(20) as workers:
    processes = (
        F(workers.map, hmmscan)
        >> F(tqdm.tqdm, total=len(ids))
        >> list
    )(protein_paths, output_paths)
    



  0%|          | 0/5961 [00:00<?, ?it/s]

  0%|          | 1/5961 [00:02<4:15:39,  2.57s/it]

  0%|          | 2/5961 [00:16<9:56:58,  6.01s/it]

  0%|          | 3/5961 [00:28<13:00:03,  7.86s/it]

  1%|          | 34/5961 [00:44<9:18:40,  5.66s/it]

  1%|          | 35/5961 [00:45<6:59:19,  4.25s/it]

  1%|          | 38/5961 [00:46<4:56:10,  3.00s/it]

  1%|          | 39/5961 [00:50<5:41:56,  3.46s/it]

  1%|          | 44/5961 [00:55<4:29:07,  2.73s/it]

  1%|          | 51/5961 [00:55<3:08:38,  1.92s/it]

  1%|          | 55/5961 [00:56<2:18:18,  1.41s/it]

  1%|          | 57/5961 [00:59<2:15:15,  1.37s/it]

  1%|          | 59/5961 [01:01<2:06:51,  1.29s/it]

  1%|          | 60/5961 [01:05<3:21:22,  2.05s/it]

  1%|          | 65/5961 [01:06<2:25:01,  1.48s/it]

  1%|          | 70/5961 [01:07<1:48:55,  1.11s/it]

  1%|          | 71/5961 [01:21<8:16:10,  5.05s/it]

  1%|▏         | 84/5961 [01:28<6:01:48,  3.69s/it]

  1%|▏         | 89/5961 [01:28<4:15:51,  2.61s/it]

  2%

 10%|▉         | 578/5961 [08:32<1:28:33,  1.01it/s]

 10%|▉         | 579/5961 [08:36<3:02:04,  2.03s/it]

 10%|▉         | 581/5961 [08:38<2:32:46,  1.70s/it]

 10%|▉         | 582/5961 [08:42<3:24:08,  2.28s/it]

 10%|▉         | 585/5961 [08:45<2:53:26,  1.94s/it]

 10%|▉         | 593/5961 [08:46<2:03:20,  1.38s/it]

 10%|▉         | 596/5961 [08:49<1:58:06,  1.32s/it]

 10%|█         | 597/5961 [08:55<4:05:19,  2.74s/it]

 10%|█         | 600/5961 [08:56<3:01:09,  2.03s/it]

 10%|█         | 601/5961 [09:00<3:45:01,  2.52s/it]

 10%|█         | 603/5961 [09:01<2:53:46,  1.95s/it]

 10%|█         | 605/5961 [09:03<2:18:19,  1.55s/it]

 10%|█         | 608/5961 [09:08<2:25:40,  1.63s/it]

 10%|█         | 609/5961 [09:15<4:43:52,  3.18s/it]

 10%|█         | 622/5961 [09:17<3:23:34,  2.29s/it]

 10%|█         | 623/5961 [09:18<2:35:39,  1.75s/it]

 10%|█         | 624/5961 [09:27<5:41:11,  3.84s/it]

 11%|█         | 632/5961 [09:29<4:07:43,  2.79s/it]

 11%|█         | 633/5961 [0

 17%|█▋        | 1037/5961 [15:31<4:52:46,  3.57s/it]

 17%|█▋        | 1038/5961 [15:34<4:42:43,  3.45s/it]

 18%|█▊        | 1047/5961 [15:36<3:22:24,  2.47s/it]

 18%|█▊        | 1048/5961 [15:38<3:33:00,  2.60s/it]

 18%|█▊        | 1049/5961 [15:39<2:43:10,  1.99s/it]

 18%|█▊        | 1051/5961 [15:44<3:00:35,  2.21s/it]

 18%|█▊        | 1053/5961 [15:48<2:44:38,  2.01s/it]

 18%|█▊        | 1055/5961 [15:51<2:39:23,  1.95s/it]

 18%|█▊        | 1073/5961 [15:52<1:51:50,  1.37s/it]

 18%|█▊        | 1075/5961 [16:01<3:11:56,  2.36s/it]

 18%|█▊        | 1078/5961 [16:03<2:34:54,  1.90s/it]

 18%|█▊        | 1079/5961 [16:04<2:04:42,  1.53s/it]

 18%|█▊        | 1080/5961 [16:23<8:58:55,  6.62s/it]

 19%|█▉        | 1140/5961 [16:23<6:12:41,  4.64s/it]

 19%|█▉        | 1149/5961 [16:34<4:51:19,  3.63s/it]

 20%|█▉        | 1171/5961 [16:38<3:26:42,  2.59s/it]

 20%|█▉        | 1190/5961 [16:45<2:33:25,  1.93s/it]

 20%|█▉        | 1191/5961 [16:46<2:09:37,  1.63s/it]

 20%|██   

 30%|███       | 1797/5961 [23:35<1:30:47,  1.31s/it]

 30%|███       | 1798/5961 [23:35<1:23:24,  1.20s/it]

 30%|███       | 1799/5961 [23:43<3:26:18,  2.97s/it]

 30%|███       | 1800/5961 [23:45<3:16:44,  2.84s/it]

 30%|███       | 1813/5961 [23:49<2:23:03,  2.07s/it]

 30%|███       | 1814/5961 [23:50<2:00:08,  1.74s/it]

 30%|███       | 1816/5961 [23:56<2:31:19,  2.19s/it]

 30%|███       | 1817/5961 [24:12<7:05:24,  6.16s/it]

 31%|███       | 1852/5961 [24:20<5:00:26,  4.39s/it]

 31%|███       | 1853/5961 [24:25<5:07:30,  4.49s/it]

 31%|███▏      | 1866/5961 [24:28<3:38:18,  3.20s/it]

 31%|███▏      | 1869/5961 [24:32<3:05:00,  2.71s/it]

 31%|███▏      | 1870/5961 [24:33<2:27:57,  2.17s/it]

 31%|███▏      | 1875/5961 [24:34<1:48:12,  1.59s/it]

 32%|███▏      | 1882/5961 [24:38<1:24:59,  1.25s/it]

 32%|███▏      | 1883/5961 [24:40<1:41:15,  1.49s/it]

 32%|███▏      | 1888/5961 [24:40<1:13:48,  1.09s/it]

 32%|███▏      | 1889/5961 [24:44<2:13:09,  1.96s/it]

 32%|███▏ 

 41%|████      | 2455/5961 [31:27<1:40:05,  1.71s/it]

 41%|████▏     | 2464/5961 [31:28<1:11:53,  1.23s/it]

 41%|████▏     | 2470/5961 [31:28<51:29,  1.13it/s]  

 41%|████▏     | 2471/5961 [31:32<1:42:14,  1.76s/it]

 42%|████▏     | 2474/5961 [31:37<1:40:42,  1.73s/it]

 42%|████▏     | 2482/5961 [31:42<1:20:19,  1.39s/it]

 42%|████▏     | 2512/5961 [31:46<58:11,  1.01s/it]  

 42%|████▏     | 2513/5961 [31:46<43:08,  1.33it/s]

 42%|████▏     | 2514/5961 [31:47<44:22,  1.29it/s]

 42%|████▏     | 2519/5961 [31:50<40:44,  1.41it/s]

 42%|████▏     | 2520/5961 [31:57<2:30:02,  2.62s/it]

 42%|████▏     | 2521/5961 [31:59<2:21:06,  2.46s/it]

 42%|████▏     | 2528/5961 [31:59<1:39:12,  1.73s/it]

 43%|████▎     | 2539/5961 [32:04<1:17:21,  1.36s/it]

 43%|████▎     | 2540/5961 [32:06<1:27:02,  1.53s/it]

 43%|████▎     | 2542/5961 [32:07<1:03:56,  1.12s/it]

 43%|████▎     | 2547/5961 [32:07<45:30,  1.25it/s]  

 43%|████▎     | 2549/5961 [32:15<1:40:02,  1.76s/it]

 43%|████▎     |

...

 61%|██████    | 3637/5961 [41:55<47:43,  1.23s/it]

 61%|██████    | 3640/5961 [42:06<1:16:05,  1.97s/it]

 61%|██████    | 3641/5961 [42:07<1:02:18,  1.61s/it]

 61%|██████    | 3648/5961 [42:14<55:17,  1.43s/it]  

 62%|██████▏   | 3668/5961 [42:15<39:00,  1.02s/it]

 62%|██████▏   | 3669/5961 [42:22<1:46:45,  2.79s/it]

 62%|██████▏   | 3671/5961 [42:23<1:17:17,  2.03s/it]

 62%|██████▏   | 3672/5961 [42:28<1:59:43,  3.14s/it]

 62%|██████▏   | 3682/5961 [42:30<1:25:19,  2.25s/it]

 62%|██████▏   | 3683/5961 [42:32<1:20:59,  2.13s/it]

 62%|██████▏   | 3684/5961 [42:34<1:15:41,  1.99s/it]

 62%|██████▏   | 3689/5961 [42:36<58:36,  1.55s/it]  

 62%|██████▏   | 3690/5961 [42:38<58:54,  1.56s/it]

 62%|██████▏   | 3691/5961 [42:40<1:06:41,  1.76s/it]

 62%|██████▏   | 3692/5961 [42:42<1:05:51,  1.74s/it]

 62%|██████▏   | 3702/5961 [42:50<55:53,  1.48s/it]  

 62%|██████▏   | 3703/5961 [42:56<1:40:21,  2.67s/it]

 62%|██████▏   | 3708/5961 [42:59<1:17:51,  2.07s/it]

 63%|██████▎   |

 70%|███████   | 4173/5961 [47:53<51:53,  1.74s/it]

 70%|███████   | 4176/5961 [47:56<44:07,  1.48s/it]

 70%|███████   | 4179/5961 [47:59<39:19,  1.32s/it]

 70%|███████   | 4180/5961 [48:01<45:38,  1.54s/it]

 70%|███████   | 4181/5961 [48:01<33:13,  1.12s/it]

 70%|███████   | 4182/5961 [48:01<25:22,  1.17it/s]

 70%|███████   | 4183/5961 [48:01<20:23,  1.45it/s]

 70%|███████   | 4184/5961 [48:04<35:48,  1.21s/it]

 70%|███████   | 4189/5961 [48:07<30:24,  1.03s/it]

 70%|███████   | 4192/5961 [48:09<27:32,  1.07it/s]

 70%|███████   | 4194/5961 [48:13<38:47,  1.32s/it]

 70%|███████   | 4196/5961 [48:14<28:43,  1.02it/s]

 70%|███████   | 4197/5961 [48:15<33:49,  1.15s/it]

 70%|███████   | 4200/5961 [48:18<30:48,  1.05s/it]

 70%|███████   | 4201/5961 [48:18<23:09,  1.27it/s]

 70%|███████   | 4202/5961 [48:25<1:15:00,  2.56s/it]

 71%|███████   | 4203/5961 [48:26<1:01:49,  2.11s/it]

 71%|███████   | 4209/5961 [48:26<43:40,  1.50s/it]  

 71%|███████   | 4210/5961 [48:26<31:33,

 76%|███████▌  | 4514/5961 [54:11<32:50,  1.36s/it]

 76%|███████▌  | 4523/5961 [54:12<23:52,  1.00it/s]

 76%|███████▌  | 4526/5961 [54:13<19:33,  1.22it/s]

 76%|███████▌  | 4529/5961 [54:28<50:09,  2.10s/it]

 76%|███████▌  | 4537/5961 [54:31<36:52,  1.55s/it]

 76%|███████▌  | 4538/5961 [54:32<34:04,  1.44s/it]

 76%|███████▌  | 4539/5961 [54:36<54:00,  2.28s/it]

 76%|███████▌  | 4541/5961 [54:37<42:37,  1.80s/it]

 76%|███████▋  | 4547/5961 [54:49<42:45,  1.81s/it]

 76%|███████▋  | 4549/5961 [54:56<57:17,  2.43s/it]

 76%|███████▋  | 4560/5961 [54:58<41:03,  1.76s/it]

 77%|███████▋  | 4566/5961 [55:02<32:28,  1.40s/it]

 77%|███████▋  | 4567/5961 [55:04<38:40,  1.66s/it]

 77%|███████▋  | 4570/5961 [55:19<1:02:37,  2.70s/it]

 77%|███████▋  | 4610/5961 [55:21<42:48,  1.90s/it]  

 77%|███████▋  | 4613/5961 [55:21<31:03,  1.38s/it]

 77%|███████▋  | 4618/5961 [55:22<22:26,  1.00s/it]

 78%|███████▊  | 4625/5961 [55:22<15:50,  1.41it/s]

 78%|███████▊  | 4627/5961 [55:24<19:15,  

 86%|████████▌ | 5100/5961 [1:01:37<42:56,  2.99s/it]

 86%|████████▋ | 5153/5961 [1:01:37<28:12,  2.10s/it]

 87%|████████▋ | 5165/5961 [1:01:41<20:42,  1.56s/it]

 87%|████████▋ | 5173/5961 [1:01:45<16:13,  1.24s/it]

 87%|████████▋ | 5179/5961 [1:01:48<13:14,  1.02s/it]

 87%|████████▋ | 5184/5961 [1:01:48<09:29,  1.36it/s]

 87%|████████▋ | 5188/5961 [1:01:51<08:52,  1.45it/s]

 87%|████████▋ | 5191/5961 [1:01:51<06:40,  1.92it/s]

 87%|████████▋ | 5193/5961 [1:01:51<05:40,  2.25it/s]

 87%|████████▋ | 5195/5961 [1:01:54<08:51,  1.44it/s]

 87%|████████▋ | 5198/5961 [1:01:54<06:49,  1.86it/s]

 87%|████████▋ | 5200/5961 [1:01:55<05:11,  2.45it/s]

 87%|████████▋ | 5202/5961 [1:01:59<10:56,  1.16it/s]

 87%|████████▋ | 5209/5961 [1:01:59<07:51,  1.60it/s]

 87%|████████▋ | 5212/5961 [1:02:01<07:54,  1.58it/s]

 87%|████████▋ | 5215/5961 [1:02:02<07:11,  1.73it/s]

 88%|████████▊ | 5219/5961 [1:02:04<06:05,  2.03it/s]

 88%|████████▊ | 5221/5961 [1:02:05<06:48,  1.81it/s]

 88%|█████

 95%|█████████▌| 5685/5961 [1:08:46<03:21,  1.37it/s]

 95%|█████████▌| 5686/5961 [1:08:47<04:26,  1.03it/s]

 95%|█████████▌| 5689/5961 [1:08:49<03:51,  1.18it/s]

 95%|█████████▌| 5690/5961 [1:08:50<03:56,  1.14it/s]

 96%|█████████▌| 5693/5961 [1:08:55<05:18,  1.19s/it]

 96%|█████████▌| 5696/5961 [1:09:02<06:42,  1.52s/it]

 96%|█████████▌| 5699/5961 [1:09:07<06:33,  1.50s/it]

 96%|█████████▌| 5701/5961 [1:09:08<05:20,  1.23s/it]

 96%|█████████▌| 5710/5961 [1:09:10<03:54,  1.07it/s]

 96%|█████████▌| 5711/5961 [1:09:11<04:01,  1.03it/s]

 96%|█████████▌| 5712/5961 [1:09:13<04:37,  1.11s/it]

 96%|█████████▌| 5714/5961 [1:09:19<06:59,  1.70s/it]

 96%|█████████▌| 5715/5961 [1:09:21<07:18,  1.78s/it]

 96%|█████████▌| 5716/5961 [1:09:26<11:33,  2.83s/it]

 96%|█████████▌| 5718/5961 [1:09:26<08:06,  2.00s/it]

 96%|█████████▌| 5719/5961 [1:09:29<08:55,  2.21s/it]

 96%|█████████▌| 5727/5961 [1:09:31<06:26,  1.65s/it]

 96%|█████████▌| 5732/5961 [1:09:33<04:40,  1.23s/it]

 96%|█████